### Modelo a usar XGBoost

In [2]:
### Para la prueba utilizaremos el modelo de XGBosst , consideranto que estamos usando un modelo que depende de varios factores
### incluyendo el tiempo

In [16]:
import joblib
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from src.data_processing import load_and_preprocess_data
import numpy as np

In [4]:
!pip install xgboost

     -------------------------------------- 150.0/150.0 MB 1.7 MB/s eta 0:00:00


#### Cargado de datos y Procesamiento

In [19]:
data_path = 'dataset_alquiler.csv'
X_train, X_test, y_train, y_test, preprocessor = load_and_preprocess_data(data_path)

Error: El archivo 'dataset_alquiler.csv' no fue encontrado.


ValueError: not enough values to unpack (expected 5, got 4)

#### Inicializacion de hiperparametros para el modelo 

In [ ]:
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror', # Función objetivo para regresión
    n_estimators=100,             # Número de árboles de decisión a construir
    learning_rate=0.1,            # Tasa de aprendizaje
    max_depth=5,                  # Profundidad máxima de cada árbol
    random_state=42               # Para reproducibilidad
)
print("\nEntrenando el modelo XGBoost...")
xgb_model.fit(X_train, y_train)
print("Modelo XGBoost entrenado.")
##### Prediccion del modelo ######
predictions = xgb_model.predict(X_test)